# Script to prepare data for YOLO training

In [13]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [14]:
# Get csv paths
path_csv_train = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/train_audible_recording_chunks.csv"
path_csv_test = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/test_audible_recording_chunks.csv"
path_csv_val = "../../Dataset acoustique insectes/CSVs morceaux audio 5s/Audible/val_audible_recording_chunks.csv"

# Get spectrograms paths
path_spectro_train = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/train_spectro"
path_spectro_test = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/test_spectro"
path_spectro_val = "../../Dataset acoustique insectes/Sélection morceaux audio 5s/Audible/val_spectro"

# Using pandas to read the CSV file into a DataFrame
df_train = pd.read_csv(path_csv_train)
df_test = pd.read_csv(path_csv_test)
df_val = pd.read_csv(path_csv_val)

In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(df_train['label'])

def clean_data(df):
    y = df['label']
    y_category = df['label_category']
    y_kingdom = df['label_kingdom']
    y_class = df['label_class']
    y_order = df['label_order']
    y_family = df['label_family']
    y_subfamily = df['label_subfamily']

    # Convertir les étiquettes en nombres
    y_encoded = label_encoder.transform(y)

    df['t1'] = df["annotation_initial_time"] - df["chunk_initial_time"]
    df['t2'] = df["annotation_final_time"] - df["chunk_initial_time"]
    df = df[['min_frequency', 'max_frequency','t1', 't2']]
    df['min_frequency'] = df['min_frequency'].astype(int)
    df['max_frequency'] = df['max_frequency'].astype(int)

    # Gestion des valeurs manquantes
    df = df.dropna()
    
    # Gestion des variables catégorielles
    df = pd.get_dummies(df)

    X = df

    return X, y_encoded

X_train, y_train = clean_data(df_train)
X_test, y_test = clean_data(df_test)
X_val, y_val = clean_data(df_val)

C:\Users\lucas\AppData\Local\Temp\ipykernel_12368\254361546.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['min_frequency'] = df['min_frequency'].astype(int)
C:\Users\lucas\AppData\Local\Temp\ipykernel_12368\254361546.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['max_frequency'] = df['max_frequency'].astype(int)
C:\Users\lucas\AppData\Local\Temp\ipykernel_12368\254361546.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [16]:
X_train

,min_frequency,max_frequency,t1,t2
0,0,375,0.355751,4.147038
1,3248,4453,0.000000,5.000000
2,3248,4453,0.000000,5.000000
3,0,375,0.346426,1.708443
4,3248,4453,0.000000,5.000000
...,...,...,...,...
10784,5007,16160,2.296621,3.300916
10785,4750,16553,4.691478,5.000000
10786,5416,17367,0.000000,0.674299
10787,5007,15776,3.399642,3.863163
